In [1]:
import os, logging
from dotenv import load_dotenv
from etl.extract import extract
from etl.transform import transform
from etl.load import load
import pandas as pd

In [2]:
load_dotenv()

DB_CONFIG = {
    'host': os.getenv("DB_HOST"),
    'user': os.getenv("DB_USER"),
    'password': os.getenv("DB_PASS"),
    'dbname': os.getenv("DB_NAME"),
    'port': os.getenv("DB_PORT")
}

def setup_logging():
    logging.basicConfig(
        filename='chicago_crime.log',
        level=logging.DEBUG,
        format='%(asctime)s - %(levelname)s - %(message)s',
        force=True  # ensures clean reconfiguration when reloaded
    )

setup_logging()

## For Local file with records from 2001 to Present

### Extract

In [3]:
from pathlib import Path
import pandas as pd

base_data_path = Path("./data")

raw_file_name = "Crimes_-_2001_to_Present_20250403.csv"
cleaned_file_name = "cleaned_Crimes_-_2001_to_Present_20250403.csv"
cleaned_subset_file_name = "cleaned_subset_2024_to_Present_20250403.csv"

In [ ]:
file_name = raw_file_name

df = pd.read_csv(base_data_path / file_name)

### Transform

In [6]:
from etl.transform import rename_columns

df = rename_columns(df)
df = transform(df)

/home/test/Documents/Workspace/ms-in-data-science-and-analytics/CIS 660/project/etl/transform.py:148: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")


In [8]:
df.to_csv(base_data_path / cleaned_file_name, index=False)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8286117 entries, 0 to 8286442
Data columns (total 21 columns):
 #   Column                Dtype         
---  ------                -----         
 0   id                    Int64         
 1   case_number           string        
 2   date                  datetime64[ns]
 3   block                 string        
 4   iucr                  string        
 5   primary_type          string        
 6   description           string        
 7   location_description  string        
 8   arrest                boolean       
 9   domestic              boolean       
 10  beat                  Int64         
 11  district              Int64         
 12  ward                  Int64         
 13  community_area        Int64         
 14  fbi_code              string        
 15  x_coordinate          float64       
 16  y_coordinate          float64       
 17  year                  Int64         
 18  updated_on            datetime64[ns]
 19  latit

### Load

Load to a local storage in csv and later import it using the COPY command for faster import

In [12]:
df.to_csv(base_data_path / cleaned_file_name, index=False)

Can also be loaded using the load function, but it can be slower

In [ ]:
df = pd.read_csv(base_data_path / cleaned_file_name)

In [ ]:
numeric_int_cols = ["id", "beat", "district", "ward", "community_area", "year"]
float_cols = ["x_coordinate", "y_coordinate", "latitude", "longitude"]
string_cols = [
    "case_number", "block", "iucr", "primary_type", "description",
    "location_description", "fbi_code"
]
bool_cols = ["arrest", "domestic"]
datetime_cols = ["date", "updated_on"]

from etl.transform import cast_column_types

df = cast_column_types(df, numeric_int_cols, float_cols, string_cols, bool_cols, datetime_cols)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8286117 entries, 0 to 8286116
Data columns (total 21 columns):
 #   Column                Dtype         
---  ------                -----         
 0   id                    Int64         
 1   case_number           string        
 2   date                  datetime64[ns]
 3   block                 string        
 4   iucr                  string        
 5   primary_type          string        
 6   description           string        
 7   location_description  string        
 8   arrest                boolean       
 9   domestic              boolean       
 10  beat                  Int64         
 11  district              Int64         
 12  ward                  Int64         
 13  community_area        Int64         
 14  fbi_code              string        
 15  x_coordinate          float64       
 16  y_coordinate          float64       
 17  year                  Int64         
 18  updated_on            datetime64[ns]
 19  

In [10]:
load(df, DB_CONFIG)

## Update Data from API continuously
The following steps are repeated daily through Airflow

### Extract

In [13]:
APP_TOKEN = os.getenv("APP_TOKEN")
records = extract(DB_CONFIG, APP_TOKEN,batch_size=10000)

In [26]:
df = pd.DataFrame(records)

In [27]:
df.loc[:5, 'community_area'] = None

In [28]:
df.head(10)

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,13717714,JJ111836,2025-01-11T16:00:00.000,055XX W GRAND AVE,4650,OTHER OFFENSE,SEX OFFENDER - FAIL TO REGISTER,JAIL / LOCK-UP FACILITY,True,False,...,37,None,26,1138788,1913480,2025,2025-04-02T15:40:14.000,41.918711651,-87.76551063,"{'latitude': '41.918711651', 'longitude': '-87..."
1,13281996,JG509881,2023-11-19T00:55:00.000,019XX W CONGRESS PKWY,041A,BATTERY,AGGRAVATED - HANDGUN,STREET,False,False,...,28,None,04B,1163642,1897746,2023,2025-04-02T15:40:14.000,41.875048478,-87.674638228,"{'latitude': '41.875048478', 'longitude': '-87..."
2,13686067,JH535382,2024-12-07T20:25:00.000,034XX S DR MARTIN LUTHER KING JR DR,0860,THEFT,RETAIL THEFT,DRUG STORE,True,False,...,4,None,06,1179474,1882305,2024,2025-04-02T15:40:14.000,41.832329279,-87.616983146,"{'latitude': '41.832329279', 'longitude': '-87..."
3,13732255,JJ128513,2021-08-01T00:00:00.000,056XX W WELLINGTON AVE,1752,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,RESIDENCE,False,True,...,31,None,17,1143220,1914849,2021,2025-04-02T15:40:14.000,41.922386649,-87.74919257,"{'latitude': '41.922386649', 'longitude': '-87..."
4,13791974,JJ201523,2025-03-15T22:30:00.000,037XX N HALSTED ST,0810,THEFT,OVER $500,TAVERN / LIQUOR STORE,False,False,...,46,None,06,1170251,1925095,2025,2025-04-02T15:40:14.000,41.949953889,-87.649572294,"{'latitude': '41.949953889', 'longitude': '-87..."
5,13743185,JJ142494,2025-02-07T22:50:00.000,058XX N KENMORE AVE,041A,BATTERY,AGGRAVATED - HANDGUN,APARTMENT,True,True,...,48,None,04B,1168188,1938974,2025,2025-04-02T15:40:14.000,41.988083216,-87.656752717,"{'latitude': '41.988083216', 'longitude': '-87..."
6,13758191,JJ160580,2023-10-19T00:00:00.000,042XX N SPAULDING AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,APARTMENT,False,False,...,33,16,11,NaN,NaN,2023,2025-04-02T15:40:14.000,NaN,NaN,NaN
7,28610,JJ158505,2025-02-22T21:53:00.000,003XX W GRAND AVE,0110,HOMICIDE,FIRST DEGREE MURDER,HOTEL,True,False,...,42,8,01A,1173615,1903832,2025,2025-04-02T15:40:14.000,41.891532981,-87.637840611,"{'latitude': '41.891532981', 'longitude': '-87..."
8,13764273,JJ168094,2025-02-09T00:00:00.000,065XX S LAFLIN ST,1152,DECEPTIVE PRACTICE,ILLEGAL USE CASH CARD,APARTMENT,False,True,...,16,67,11,1167464,1861270,2025,2025-04-02T15:40:14.000,41.774873231,-87.661652469,"{'latitude': '41.774873231', 'longitude': '-87..."
9,13767127,JJ171658,2025-02-27T18:00:00.000,056XX S RICHMOND ST,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,RESIDENCE,False,False,...,14,63,11,1157674,1867092,2025,2025-04-02T15:40:14.000,41.791053851,-87.697383548,"{'latitude': '41.791053851', 'longitude': '-87..."


### Transform

In [29]:
df = transform(df)

In [30]:
df.head(10)

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude
0,13717714,JJ111836,2025-01-11 16:00:00,055XX W GRAND AVE,4650,OTHER OFFENSE,SEX OFFENDER - FAIL TO REGISTER,JAIL / LOCK-UP FACILITY,True,False,...,25,37,19,26,1138788.0,1913480.0,2025,2025-04-02 15:40:14,41.918712,-87.765511
1,13281996,JG509881,2023-11-19 00:55:00,019XX W CONGRESS PKWY,041A,BATTERY,AGGRAVATED - HANDGUN,STREET,False,False,...,12,28,28,04B,1163642.0,1897746.0,2023,2025-04-02 15:40:14,41.875048,-87.674638
2,13686067,JH535382,2024-12-07 20:25:00,034XX S DR MARTIN LUTHER KING JR DR,0860,THEFT,RETAIL THEFT,DRUG STORE,True,False,...,2,4,35,06,1179474.0,1882305.0,2024,2025-04-02 15:40:14,41.832329,-87.616983
3,13732255,JJ128513,2021-08-01 00:00:00,056XX W WELLINGTON AVE,1752,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,RESIDENCE,False,True,...,25,31,19,17,1143220.0,1914849.0,2021,2025-04-02 15:40:14,41.922387,-87.749193
4,13791974,JJ201523,2025-03-15 22:30:00,037XX N HALSTED ST,0810,THEFT,OVER $500,TAVERN / LIQUOR STORE,False,False,...,19,46,6,06,1170251.0,1925095.0,2025,2025-04-02 15:40:14,41.949954,-87.649572
5,13743185,JJ142494,2025-02-07 22:50:00,058XX N KENMORE AVE,041A,BATTERY,AGGRAVATED - HANDGUN,APARTMENT,True,True,...,20,48,77,04B,1168188.0,1938974.0,2025,2025-04-02 15:40:14,41.988083,-87.656753
6,13758191,JJ160580,2023-10-19 00:00:00,042XX N SPAULDING AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,APARTMENT,False,False,...,17,33,16,11,NaN,NaN,2023,2025-04-02 15:40:14,NaN,NaN
7,28610,JJ158505,2025-02-22 21:53:00,003XX W GRAND AVE,0110,HOMICIDE,FIRST DEGREE MURDER,HOTEL,True,False,...,18,42,8,01A,1173615.0,1903832.0,2025,2025-04-02 15:40:14,41.891533,-87.637841
8,13764273,JJ168094,2025-02-09 00:00:00,065XX S LAFLIN ST,1152,DECEPTIVE PRACTICE,ILLEGAL USE CASH CARD,APARTMENT,False,True,...,7,16,67,11,1167464.0,1861270.0,2025,2025-04-02 15:40:14,41.774873,-87.661652
9,13767127,JJ171658,2025-02-27 18:00:00,056XX S RICHMOND ST,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,RESIDENCE,False,False,...,8,14,63,11,1157674.0,1867092.0,2025,2025-04-02 15:40:14,41.791054,-87.697384


### Load

In [31]:
load(df, DB_CONFIG)